In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta
import re

Test de webscrapping réalisé sur le site internet du Tour de France. 

Dans ce test, j'ai récupéré le classement général final.
Pour chaque coureur, j'ai récupéré :
- son rang dans le classement
- son nom
- son équipe
- le temps total qui correspond au cumul des temps réalisés à chaque étape du Tour de France.

Une fois ces informations récupérées, j'ai converti le temps total en timedelta pour pouvoir ensuite calculer l'écart de temps entre le 1er au classement et chaque coureur. J'aurais pu récupérer l'écart directement sur le site mais j'ai préféré le calculer à partir du temps total.

In [66]:
url = 'https://www.letour.fr/fr/classements/etape-21'
response = requests.get(url)
soup = BeautifulSoup(response.text)

positions_classement = soup.findAll('td', class_='rankingTables__row__position is-alignCenter')
list_positions = []

for pos in positions_classement:
    val = pos.span.text
    list_positions.append(int(val))

noms_coureurs = soup.findAll('a', class_='rankingTables__row__profile--name')
list_noms_coureurs = []

for nom in noms_coureurs:
    coureur = nom.text.strip()
    list_noms_coureurs.append(coureur)

classes_noms_equipes = soup.findAll('td', class_='break-line team')
list_noms_equipes = []

for classe in classes_noms_equipes:
    balise_a = classe.find('a')
    equipe = balise_a.text.strip()
    list_noms_equipes.append(equipe)

temps_total_coureurs = soup.findAll('td', class_='is-alignCenter time')
list_temps_total = []

for temps in temps_total_coureurs:
    val = temps.text.strip()
    list_temps_total.append(val)

list_temps_total_cleaned = [temps for temps in list_temps_total if temps.startswith('8')]

In [99]:
classement_general = pd.DataFrame(columns = ['Rang', 'Coureur', 'Equipe', 'Temps'])
classement_general['Rang'] = list_positions
classement_general['Coureur'] = list_noms_coureurs
classement_general['Equipe'] = list_noms_equipes
classement_general['Temps'] = list_temps_total_cleaned
classement_general.head(15)

,Rang,Coureur,Equipe,Temps
0,1,J. VINGEGAARD,JUMBO-VISMA,82h 05' 42''
1,2,T. POGAČAR,UAE TEAM EMIRATES,82h 13' 11''
2,3,A. YATES,UAE TEAM EMIRATES,82h 16' 38''
3,4,S. YATES,TEAM JAYCO ALULA,82h 18' 05''
4,5,C. RODRIGUEZ CANO,INEOS GRENADIERS,82h 18' 59''
5,6,P. BILBAO LOPEZ,BAHRAIN VICTORIOUS,82h 19' 09''
6,7,J. HINDLEY,BORA - HANSGROHE,82h 20' 26''
7,8,F. GALL,AG2R CITROEN TEAM,82h 21' 51''
8,9,D. GAUDU,GROUPAMA - FDJ,82h 28' 50''
9,10,G. MARTIN,COFIDIS,82h 32' 12''


In [89]:
def convert_time_to_timedelta(time):
    resultats = re.findall(r'(\d+)h (\d+)\' (\d+)\'\'', time)
    h, m, s = map(int, resultats[0])
    delta = timedelta(hours=h, minutes=m, seconds=s)
    return delta

In [101]:
classement_general['Duree'] = classement_general.apply(lambda row: convert_time_to_timedelta(row['Temps']), axis=1)
classement_general['Ecart'] = classement_general.apply(lambda row: row['Duree'] - classement_general.iloc[0,4], axis=1)
classement_general

,Rang,Coureur,Equipe,Temps,Duree,Ecart
0,1,J. VINGEGAARD,JUMBO-VISMA,82h 05' 42'',3 days 10:05:42,0 days 00:00:00
1,2,T. POGAČAR,UAE TEAM EMIRATES,82h 13' 11'',3 days 10:13:11,0 days 00:07:29
2,3,A. YATES,UAE TEAM EMIRATES,82h 16' 38'',3 days 10:16:38,0 days 00:10:56
3,4,S. YATES,TEAM JAYCO ALULA,82h 18' 05'',3 days 10:18:05,0 days 00:12:23
4,5,C. RODRIGUEZ CANO,INEOS GRENADIERS,82h 18' 59'',3 days 10:18:59,0 days 00:13:17
...,...,...,...,...,...,...
145,146,A. EDMONDSON,TEAM DSM - FIRMENICH,87h 50' 21'',3 days 15:50:21,0 days 05:44:39
146,147,F. FRISON,LOTTO DSTNY,88h 01' 02'',3 days 16:01:02,0 days 05:55:20
147,148,Y. FEDOROV,ASTANA QAZAQSTAN TEAM,88h 02' 19'',3 days 16:02:19,0 days 05:56:37
148,149,C. BOL,ASTANA QAZAQSTAN TEAM,88h 03' 26'',3 days 16:03:26,0 days 05:57:44
